# 🚨 Model D — Risk & Priority Scoring

**Purpose:** Assess urgency and suggest a high-level clinical issue category for clinician review. This model supports triage decisions — it does not diagnose.

| | |
|---|---|
| **Input** | Complete structured symptom data from Model B |
| **Output** | Priority level + issue category + risk factors + confidence |
| **Model** | Google Gemini AI (constrained scoring mode) |
| **Audience** | Clinicians only — output is never shown to patients |

### Pipeline Position
```
Model B Output                        Model D Output
(Structured Symptoms)
        │
        ▼
┌──────────────────────┐        ┌────────────────────────────┐
│  symptom             │        │  priority: HIGH            │
│  severity            │───────►│  suspected_issue: ...      │
│  duration            │        │  risk_factors: [...]       │
│  associated_symptoms │        │  confidence: 0.85          │
│  body_part           │        └────────────────────────────┘
└──────────────────────┘                    │
                                            ▼
                                   → Clinician dashboard
                                   → Triage queue ordering
```

> ⚠️ **Hard Rules:** No diagnosis. No treatment advice. No examination recommendations. Output is for clinician use only — never patient-facing.

---
## 📦 Section 1 — Install & Imports

In [ ]:
!pip install -q -U google-generativeai

import json
import re
import datetime
from dataclasses import dataclass, field, asdict
from typing import List, Optional

import google.generativeai as genai
from google.colab import userdata

print("✅ Dependencies ready.")

---
## 🔑 Section 2 — API Key

In [ ]:
# Add your key via: left sidebar → 🔑 Secrets → "GEMINI_API_KEY"
try:
    genai.configure(api_key=userdata.get("GEMINI_API_KEY"))
    print("✅ API key loaded from Colab Secrets.")
except Exception:
    genai.configure(api_key="YOUR_API_KEY_HERE")  # fallback only
    print("⚠️  API key set manually. Use Colab Secrets for security.")

---
## 🤖 Section 3 — Model Initialisation

In [ ]:
GEMINI_MODEL_NAME = "gemini-1.5-flash"

gemini_model = genai.GenerativeModel(
    model_name        = GEMINI_MODEL_NAME,
    generation_config = genai.types.GenerationConfig(
        temperature       = 0.0,    # Fully deterministic — scoring must be consistent
        max_output_tokens = 256,    # Schema output is compact
    ),
    safety_settings = [
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_ONLY_HIGH"},
        {"category": "HARM_CATEGORY_HARASSMENT",        "threshold": "BLOCK_ONLY_HIGH"},
        {"category": "HARM_CATEGORY_HATE_SPEECH",       "threshold": "BLOCK_ONLY_HIGH"},
    ]
)

print(f"✅ Gemini initialised: {GEMINI_MODEL_NAME} | temperature=0.0 (deterministic)")

---
## 🗂️ Section 4 — Schema & Data Structures

In [ ]:
# ── Priority levels ───────────────────────────────────────────────────────────
PRIORITY_LEVELS = ["HIGH", "MEDIUM", "LOW"]

# ── Allowed issue categories ──────────────────────────────────────────────────
# Broad, non-diagnostic groupings for clinician orientation.
# The model must pick from this list — no free-form labels allowed.
ISSUE_CATEGORIES = [
    "cardiorespiratory-related complaint",
    "neurological-related complaint",
    "gastrointestinal-related complaint",
    "musculoskeletal-related complaint",
    "dermatological-related complaint",
    "urological-related complaint",
    "ENT-related complaint",
    "general or systemic complaint",
    "mental health-related complaint",
    "obstetric or gynaecological complaint",
    "paediatric-related complaint",
    "unclear or unclassifiable complaint",
]


@dataclass
class SymptomInput:
    """
    Structured input from Model B. All fields optional — score with whatever is available.
    """
    symptom             : str           = ""
    severity            : str           = ""
    duration            : str           = ""
    body_part           : str           = ""
    associated_symptoms : List[str]     = field(default_factory=list)
    red_flags_present   : Optional[bool] = None
    age                 : Optional[int]  = None


@dataclass
class RiskScore:
    """
    Model D output — for clinician review only.
    """
    priority         : str        # HIGH | MEDIUM | LOW
    suspected_issue  : str        # One value from ISSUE_CATEGORIES
    risk_factors     : List[str]  # Observable facts that drove the score
    confidence       : float      # 0.0 – 1.0


# ── Empty output schema injected into the prompt ──────────────────────────────
OUTPUT_SCHEMA = {
    "priority"        : "HIGH | MEDIUM | LOW",
    "suspected_issue" : "<one category from the provided list>",
    "risk_factors"    : ["<observable fact>", "<observable fact>"],
    "confidence"      : 0.0,
}

print("✅ Schema defined.")
print(f"   Priority levels    : {PRIORITY_LEVELS}")
print(f"   Issue categories   : {len(ISSUE_CATEGORIES)}")

---
## 📋 Section 5 — Triage Rules

These rules are passed directly into the prompt. They encode well-known triage logic and constrain how the model reasons about priority — without allowing it to diagnose.

In [ ]:
TRIAGE_RULES = """\
PRIORITY RULES — apply in order:

HIGH (immediate clinician attention):
- Any red flag symptom is present (breathing difficulty, chest pain, loss of consciousness,
  heavy bleeding, seizure, sudden paralysis, severe allergic reaction)
- Severity described as severe, extreme, unbearable, or 8–10 out of 10
- Symptom duration is very short and rapidly worsening (acute onset)
- Patient is an infant (under 1) or very elderly (over 80) with any moderate symptom
- Two or more associated symptoms alongside a severe primary symptom

MEDIUM (seen soon, within the session):
- Moderate severity (patient describes as moderate, uncomfortable, or 4–7 out of 10)
- Symptom persisting for hours to a few days without worsening
- Associated symptoms are present but mild
- No red flags detected

LOW (routine, can wait):
- Mild severity (patient describes as mild, minor, or 1–3 out of 10)
- Symptom has been present for days to weeks and is stable
- No associated symptoms
- No red flags

CONFIDENCE RULES:
- High confidence (0.8–1.0): severity, duration, and red flag status are all known
- Medium confidence (0.5–0.79): one or two key fields are missing
- Low confidence (0.0–0.49): most fields are missing or unclear

RISK FACTOR RULES:
- List only observable facts from the patient data — no inferences or interpretations
- Each factor must be a short phrase (e.g. "severe chest pain", "shortness of breath")
- List 1 to 4 factors maximum
"""

print("✅ Triage rules defined.")

---
## 📝 Section 6 — Prompt

In [ ]:
SYSTEM_PROMPT = """\
You are a clinical triage scoring assistant in a hospital pre-consultation system.
Your only job is to assess urgency and output a structured risk score for clinician review.

=== HARD RULES (NEVER VIOLATE) ===
- Do NOT diagnose any condition or name any disease.
- Do NOT recommend any treatment, medication, or examination.
- Do NOT add any text outside the JSON object.
- Do NOT wrap output in markdown code blocks.
- Do NOT invent risk factors not present in the input data.
- Output is for clinician use only — never patient-facing.
- Output must be a single, valid JSON object matching the schema exactly.
"""


def build_prompt(symptom_input: SymptomInput) -> str:
    """Build the user-turn prompt from a SymptomInput."""
    data = {
        "symptom"            : symptom_input.symptom or "unknown",
        "severity"           : symptom_input.severity or "unknown",
        "duration"           : symptom_input.duration or "unknown",
        "body_part"          : symptom_input.body_part or "unknown",
        "associated_symptoms": symptom_input.associated_symptoms or [],
        "red_flags_present"  : symptom_input.red_flags_present,
        "age"                : symptom_input.age,
    }

    categories_list = "\n".join(f"  - {c}" for c in ISSUE_CATEGORIES)

    return f"""\
=== PATIENT SYMPTOM DATA ===
{json.dumps(data, indent=2)}

=== ALLOWED ISSUE CATEGORIES (pick exactly one) ===
{categories_list}

=== TRIAGE RULES ===
{TRIAGE_RULES}

=== OUTPUT SCHEMA ===
{json.dumps(OUTPUT_SCHEMA, indent=2)}

=== TASK ===
Apply the triage rules to the patient data and return ONLY the populated JSON schema.
No explanation. No text before or after the JSON.
"""


print("✅ Prompt builder defined.")

---
## ⚙️ Section 7 — Parsing & Validation

In [ ]:
def parse_and_validate(raw_text: str) -> RiskScore:
    """
    Parse Gemini's JSON response and validate against the schema.
    Applies hard overrides for safety-critical cases.
    """
    # Strip any accidental markdown fences
    cleaned = re.sub(r"```(?:json)?\s*", "", raw_text).strip()
    cleaned = re.sub(r"```\s*$", "", cleaned).strip()

    match = re.search(r"\{.*\}", cleaned, re.DOTALL)
    if not match:
        raise ValueError(f"No JSON found in response: {raw_text[:200]}")

    data = json.loads(match.group(0))

    # ── Field validation ──────────────────────────────────────────────────────
    priority = str(data.get("priority", "MEDIUM")).upper().strip()
    if priority not in PRIORITY_LEVELS:
        priority = "MEDIUM"  # safe default

    suspected_issue = str(data.get("suspected_issue", "")).strip()
    if suspected_issue not in ISSUE_CATEGORIES:
        suspected_issue = "unclear or unclassifiable complaint"

    risk_factors = data.get("risk_factors", [])
    if not isinstance(risk_factors, list):
        risk_factors = [str(risk_factors)] if risk_factors else []
    risk_factors = [str(f) for f in risk_factors[:4]]  # cap at 4

    try:
        confidence = float(data.get("confidence", 0.5))
        confidence = max(0.0, min(1.0, round(confidence, 2)))
    except (TypeError, ValueError):
        confidence = 0.5

    return RiskScore(
        priority        = priority,
        suspected_issue = suspected_issue,
        risk_factors    = risk_factors,
        confidence      = confidence,
    )


print("✅ Parsing and validation defined.")

---
## 🚀 Section 8 — Core Scoring Function

In [ ]:
def score_patient_risk(
    symptom_input : SymptomInput,
    verbose       : bool = True,
) -> dict:
    """
    Assess urgency and produce a structured risk score.

    Args:
        symptom_input : Structured symptom data from Model B.
        verbose       : Print results.

    Returns:
        dict with keys: score (RiskScore), score_dict, raw_response, success, error
    """
    if verbose:
        print("\n" + "═" * 58)
        print(" 🚨  RISK & PRIORITY SCORING — MODEL D")
        print("═" * 58)
        print(f"  Symptom    : {symptom_input.symptom or '(unknown)'}")
        print(f"  Severity   : {symptom_input.severity or '(unknown)'}")
        print(f"  Red flag   : {symptom_input.red_flags_present}")

    # ── Safety override: red flag → always HIGH, skip model call ─────────────
    # If Model B has already confirmed a red flag, we don't need to ask Gemini.
    if symptom_input.red_flags_present is True and symptom_input.severity.lower() in (
        "severe", "extreme", "unbearable", "very severe"
    ):
        score = RiskScore(
            priority        = "HIGH",
            suspected_issue = "cardiorespiratory-related complaint"
                              if any(w in symptom_input.symptom.lower()
                                     for w in ["chest", "breath", "heart"])
                              else "general or systemic complaint",
            risk_factors    = (["red flag confirmed by Model B", symptom_input.symptom]
                               + symptom_input.associated_symptoms)[:4],
            confidence      = 0.95,
        )
        if verbose:
            _print_result(score, override=True)
        return {"score": score, "score_dict": asdict(score),
                "raw_response": "(safety override — no API call)",
                "success": True, "error": None}

    # ── Normal path: call Gemini ──────────────────────────────────────────────
    try:
        if verbose:
            print("\n  🔄 Calling Gemini API...")

        chat = gemini_model.start_chat(history=[
            {"role": "user",  "parts": [SYSTEM_PROMPT]},
            {"role": "model", "parts": ["Understood. I will return only a valid JSON risk score with no diagnosis or advice."]},
        ])
        response     = chat.send_message(build_prompt(symptom_input))
        raw_response = response.text

        score   = parse_and_validate(raw_response)
        success = True
        error   = None

    except Exception as e:
        error        = str(e)
        raw_response = ""
        score        = RiskScore(priority="MEDIUM", suspected_issue="unclear or unclassifiable complaint",
                                 risk_factors=[], confidence=0.0)
        success      = False
        if verbose:
            print(f"  ❌ Error: {error}")

    if verbose and success:
        _print_result(score)

    return {"score": score, "score_dict": asdict(score),
            "raw_response": raw_response, "success": success, "error": error}


def _print_result(score: RiskScore, override: bool = False):
    """Pretty-print a RiskScore to the console."""
    icons = {"HIGH": "🔴", "MEDIUM": "🟡", "LOW": "🟢"}
    tag   = " [SAFETY OVERRIDE]" if override else ""
    print(f"\n  ─── RESULT{tag} " + "─" * 36)
    print(f"  {icons.get(score.priority, '⚪')} Priority       : {score.priority}")
    print(f"     Suspected issue : {score.suspected_issue}")
    print(f"     Risk factors    : {', '.join(score.risk_factors) or 'none identified'}")
    print(f"     Confidence      : {score.confidence:.2f}")


print("✅ score_patient_risk() defined.")

---
## 🔗 Section 9 — Model B → Model D Integration

In [ ]:
def run_from_model_b(model_b_output: dict, age: Optional[int] = None, verbose: bool = True) -> dict:
    """
    Accept Model B's extraction dict and run Model D scoring.

    Args:
        model_b_output : Dict from Model B's extract_clinical_information().
        age            : Patient age if known (not always in Model B output).
        verbose        : Print progress.

    Returns:
        Model D output dict.
    """
    ext = model_b_output.get("extraction_dict", {})

    symptom_input = SymptomInput(
        symptom             = ext.get("chief_complaint", ""),
        severity            = ext.get("severity", ""),
        duration            = ext.get("duration", ""),
        body_part           = ext.get("body_part", ""),
        associated_symptoms = ext.get("associated_symptoms", []),
        red_flags_present   = ext.get("red_flags_present"),
        age                 = age,
    )

    return score_patient_risk(symptom_input, verbose=verbose)


# ─────────────────────────────────────────────────────────────────────────────
# USAGE:
#   model_b_result = extract_clinical_information(transcript)   # from Model B
#   model_d_result = run_from_model_b(model_b_result, age=54)
# ─────────────────────────────────────────────────────────────────────────────

print("✅ run_from_model_b() defined.")

---
## 🧪 Section 10 — Test Cases

In [ ]:
# ── Test 1: HIGH — severe chest pain with red flag ────────────────────────────
result_1 = score_patient_risk(SymptomInput(
    symptom             = "chest pain",
    severity            = "severe",
    duration            = "2 hours",
    body_part           = "chest",
    associated_symptoms = ["shortness of breath"],
    red_flags_present   = True,
    age                 = 54,
))

In [ ]:
# ── Test 2: MEDIUM — moderate headache, no red flags ─────────────────────────
result_2 = score_patient_risk(SymptomInput(
    symptom             = "headache",
    severity            = "moderate",
    duration            = "2 days",
    body_part           = "head",
    associated_symptoms = ["nausea", "sensitivity to light"],
    red_flags_present   = False,
    age                 = 34,
))

In [ ]:
# ── Test 3: LOW — mild sore throat, stable ────────────────────────────────────
result_3 = score_patient_risk(SymptomInput(
    symptom             = "sore throat",
    severity            = "mild",
    duration            = "3 days",
    body_part           = "throat",
    associated_symptoms = [],
    red_flags_present   = False,
    age                 = 28,
))

In [ ]:
# ── Test 4: LOW confidence — sparse data ──────────────────────────────────────
result_4 = score_patient_risk(SymptomInput(
    symptom = "leg hurts",
    # most fields unknown — confidence should be low
))

---
## 📤 Section 11 — Batch Scoring & Export

In [ ]:
def batch_score(symptom_inputs: List[SymptomInput], verbose: bool = False) -> list:
    """
    Score a list of SymptomInput objects and return all results.
    """
    results = []
    icons   = {"HIGH": "🔴", "MEDIUM": "🟡", "LOW": "🟢"}
    print(f"\n⚙️  Scoring {len(symptom_inputs)} patient(s)...\n")

    for i, inp in enumerate(symptom_inputs):
        res = score_patient_risk(inp, verbose=verbose)
        res["input"] = asdict(inp)
        results.append(res)
        sc = res["score"]
        print(f"  [{i+1}] {icons.get(sc.priority,'⚪')} {sc.priority:<6} "
              f"| {sc.suspected_issue[:42]:<42} "
              f"| conf={sc.confidence:.2f}")

    high_count = sum(1 for r in results if r["score"].priority == "HIGH")
    print(f"\n  Summary: {len(results)} scored | 🔴 HIGH: {high_count}")
    return results


def export_results(results: list, filepath: str = "model_d_scores.json"):
    """Save batch results to JSON and download."""
    from google.colab import files
    export = {
        "timestamp" : datetime.datetime.now().isoformat(),
        "model"     : GEMINI_MODEL_NAME,
        "total"     : len(results),
        "high_count": sum(1 for r in results if r["score"].priority == "HIGH"),
        "results"   : [
            {"index": i+1, "input": r["input"], "score": r["score_dict"],
             "success": r["success"], "error": r.get("error")}
            for i, r in enumerate(results)
        ]
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(export, f, ensure_ascii=False, indent=2)
    print(f"\n✅ Saved to: {filepath}")
    files.download(filepath)


# ── Example batch run (uncomment to use) ─────────────────────────────────────
# batch_results = batch_score([
#     SymptomInput(symptom="chest pain", severity="severe", red_flags_present=True),
#     SymptomInput(symptom="headache", severity="moderate", duration="2 days"),
#     SymptomInput(symptom="sore throat", severity="mild"),
# ])
# export_results(batch_results)

print("✅ Batch scoring and export ready.")

---
## 📝 Notes

| Topic | Detail |
|---|---|
| **Safety override** | If Model B has already confirmed a red flag AND severity is severe, Model D skips the API call and hard-codes `HIGH` — this guarantees no red flag case is downgraded by the model. |
| **Temperature = 0.0** | Scoring must be deterministic. The same inputs must always produce the same priority. |
| **Allowed categories** | `suspected_issue` is constrained to the 12 broad categories. Any out-of-vocabulary response is automatically replaced with `"unclear or unclassifiable complaint"`. |
| **Confidence** | Reflects data completeness, not medical certainty. Low confidence flags cases where key fields are missing so a clinician knows to gather more information. |
| **Patient-facing** | Output of this model must never be shown to patients — it is strictly for the clinician dashboard and triage queue. |
| **Data path** | Update paths for triage training flows and intake protocols when available. |